In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords


In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
df=pd.read_csv("/content/sqli.csv", encoding='utf-16')
df.head()

,Sentence,Label
0,select * from users where id = 1 or ( \ ) =...,1
1,select * from users where id = 1 or ( \. ) ...,1
2,select * from users where id = 1 or ( \+ ) ...,1
3,select * from users where id = 1 or ( 1 ) =...,1
4,select * from users where id = 1 or ( \+ ) ...,1


In [7]:
df = df.dropna()

In [8]:
X = df['Sentence'].values
y = df['Label']

In [9]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)

In [10]:

X_pad = pad_sequences(X_seq, padding='post')

In [11]:
input_length = X.shape[0]  # Get the size of the first (and only) dimension

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.3, random_state=42,shuffle=True)

In [13]:
vocab_size = len(tokenizer.word_index) + 1  # Ensure vocab size includes padding and OOV token
embedding_dim = 100

In [14]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_length))
model.add(LSTM(128, return_sequences=True))  # First LSTM layer
model.add(Dropout(0.3))
model.add(LSTM(64))  # Second LSTM layer
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [15]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 120s 2s/step - accuracy: 0.7356 - loss: 0.5810 - val_accuracy: 0.7404 - val_loss: 0.5761
Epoch 2/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.7614 - loss: 0.5503 - val_accuracy: 0.7404 - val_loss: 0.5767
Epoch 3/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 139s 2s/step - accuracy: 0.7583 - loss: 0.5531 - val_accuracy: 0.7404 - val_loss: 0.5728
Epoch 4/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.7651 - loss: 0.5464 - val_accuracy: 0.7404 - val_loss: 0.5730
Epoch 5/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.7683 - loss: 0.5420 - val_accuracy: 0.7404 - val_loss: 0.5740


In [17]:
# Evaluate on the test data
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convert predictions to 0 or 1

48/48 ━━━━━━━━━━━━━━━━━━━━ 17s 348ms/step


In [18]:
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.7404479578392622
